In [135]:
%matplotlib inline
import matplotlib.pyplot as plt
import pyneb as pn

import numpy as np
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
from scipy.interpolate import Akima1DInterpolator
from scipy import optimize as opt
import sys
import emcee
import numpy as np
from scipy.optimize import curve_fit
from astropy.table import Table

import numpy as np
from astropy.io import fits
import time
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from scipy import signal
import matplotlib.pyplot as plt
import warnings
import pandas as pd

In [136]:
h_beta = 4861.333
h_alpha = 6562.819
h_gamma = 4340.1

In [137]:
def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

# Line model including continuum
def line(x, b):
    return np.ones(len(x)) * b

def line_model(x, A, mu, sigma, b):
    return gaussian(x, A, mu, sigma) + line(x, b)

def log_likelihood(theta, x, y, yerr):
    model = line_model(x, *theta)
    lnL = -0.5 * np.sum((y - model)**2 / yerr**2)
    return lnL

def log_prior(theta, wave_center, Amp_max):
    A, mu, sigma, b = theta
    left_mu = wave_center - 5
    right_mu = wave_center + 5
    min_A = 0
    max_A = Amp_max * 2
    sigma_window_left = 0.0001
    sigma_window_right = 50
    if (0 < A < max_A) & (left_mu <= mu <= right_mu) & (sigma_window_left <= sigma < sigma_window_right) & (b > 0):
        return 0.0
    else:
        return -np.inf

def log_probability(theta, x, y, yerr, wave_center, Amp_max):
    lp = log_prior(theta, wave_center, Amp_max)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# Function to fit the Hα and Hβ lines
def fitting_line(wave, flux, flux_err, line_center, window_wavelength, diagnose=False):
    min_window = line_center - window_wavelength
    max_window = line_center + window_wavelength
    indx = (wave >= min_window) & (wave <= max_window)
    
    spec_window = flux[indx]
    wave_window = wave[indx]
    err_spec_window = flux_err[indx]
    
    # Initial guess for the curve fit
    guess_A = np.abs(np.max(spec_window))
    guess_mu = line_center
    guess_sigma = 1.0
    guess_b = np.abs(np.median(spec_window))
    
    low_bounds = [0, min_window, 0, -guess_b]
    high_bounds = [2 * guess_A, max_window, 100, 2 * guess_b]
    popt, _ = curve_fit(line_model, wave_window, spec_window, p0=[guess_A, guess_mu, guess_sigma, guess_b],
                        bounds=(low_bounds, high_bounds))
    if diagnose == True: 
        plotting_code = line_model(wave_window,popt[0],popt[1],popt[2],popt[3])
#         plt.figure()
#         plt.plot(wave_window,plotting_code,c='cadetblue',label='model')
#         plt.plot(wave_window,spec_window,c='purple',label='data')
#         plt.axvline(min_window)
#         plt.axvline(max_window)

#         plt.legend()
#         plt.show()
    fluxes_emcee = popt[0] * popt[2] * np.sqrt(2 * np.pi)
    
   # return fluxes_emcee
    return popt



def emcee_fit(wave, flux, flux_err, line_center, window_wavelength, 
                 diagnose = False,save=True, filename = 'Emcee_Chains_Galaxy.txt'):
    
    result = fitting_line(wave, flux, flux_err, line_center, window_wavelength, diagnose=True)
    
    #getting the results from the initial fit to then pass into emcee
    guess_A = result[0]
    guess_mu = result[1]
    guess_sigma = result[2]
    guess_b = result[3]
    
    
    #making walkers so that we can use emcee to explore the parameter space
    #centered on the best results from minimization
    amp_jump = np.random.normal(loc = guess_A,            #centered on best A from minimization
                                scale = guess_A/10,       #can wander 1/10 of the value of A
                                size = 32).reshape(-1, 1) 
    
    wavelength_jump = np.random.normal(loc = guess_mu,    #centered on best mu from minimization
                                       scale = .005,      #can wander +/- 0.005 microns 
                                       size = 32).reshape(-1, 1)
    
    sigma_jump = np.random.normal(loc = guess_sigma, scale = .002, size = 32).reshape(-1, 1)

    
    powerb = np.log10(np.abs(guess_b))
    
    b_jump = np.random.normal(loc = guess_b, scale = 1*10**powerb, size = 32).reshape(-1, 1)

    
    #################
    # Diagnostic plotting to see if the parameters were jumping to large values
    # The should be concentrated near their best fit results values
    #################
    if diagnose == True:
        print('Checking the Walker Jumps')
        fig, ax = plt.subplots(nrows = 2, ncols = 2, constrained_layout = True)
        
        ax[0, 0].hist(amp_jump)
        ax[0, 0].set_xlabel('Amplitude')
        
        ax[0, 1].hist(wavelength_jump)
        ax[0, 1].set_xlabel(r'$\mu$')
        
        ax[1, 0].hist(sigma_jump)
        ax[1, 0].set_xlabel(r'$\sigma$')
        
        ax[1, 1].hist(b_jump)
        ax[1, 1].set_xlabel('b')
        
        plt.show()
    

    #stacking along the columns
    starting_walkers = np.hstack((amp_jump,
                                  wavelength_jump, 
                                  sigma_jump, 
                                  #m_jump, 
                                  b_jump))

    #initializing window for emcee around the best result mu
    emcee_window = window_wavelength #in units of microns
    emcee_indx = np.where((wave >= (line_center - emcee_window)) & 
                          (wave <= (line_center + emcee_window)))[0]

    #emcee subsections
    emcee_spec = flux[emcee_indx]
    emcee_wave = wave[emcee_indx]
    emcee_err = flux_err[emcee_indx]
    
    
    ###########
    #NOTE:
    #need to change output name everytime you run otherwise it will overwrite
    ###########
    
    #saves the input emcee spectra
    emcee_spec_matrix = np.c_[emcee_wave, emcee_spec, emcee_err]
    #np.savetxt(f'Emcee_Spectra.txt', emcee_spec_matrix)

    #initializing walker positions
    pos = starting_walkers
    nwalkers, ndim = pos.shape

    #initializing sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, 
                                    args=(emcee_wave, emcee_spec, emcee_err, guess_mu, guess_A), 
                                    moves = [(emcee.moves.DEMove(), 0.5),
                                             (emcee.moves.DESnookerMove(), 0.5)])

    #running it
    sampler.run_mcmc(pos, 3000, progress=False)

    #getting values back
    #samples = sampler.get_chain()
    flat_samples = sampler.get_chain(flat=True)
    LnL_chain = sampler.flatlnprobability
    burn_in = 1000 
    
    emcee_df = pd.DataFrame()
    emcee_df['A'] = flat_samples[burn_in:, 0]
    emcee_df['mu'] = (flat_samples[burn_in:, 1] ) 
    emcee_df['sigma'] = flat_samples[burn_in:, 2]
    emcee_df['b'] = flat_samples[burn_in:, 3]
    emcee_df['LnL'] = LnL_chain[burn_in:]
    
    emcee_df = emcee_df[np.isfinite(emcee_df.LnL.values)]
    
  #  conversion_factor = speed_of_light / ((emcee_df['mu']* um_to_ang)**2)
#    fluxes_emcee = (emcee_df['A']*conversion_factor) * (emcee_df['sigma']*um_to_ang) * np.sqrt(2 * np.pi)
   
    emcee_df['Fluxes'] = fluxes_emcee 
    
    if diagnose == True:
        
        print('Checking Prameter Posterior Distributions')
        fig, ax = plt.subplots(nrows = 2, ncols = 2, constrained_layout = True)
        
        emcee_df.A.hist(ax = ax[0, 0])
        emcee_df.mu.hist(ax = ax[0, 1])
        emcee_df.sigma.hist(ax = ax[1, 0])
        #emcee_df.m.hist(ax = ax[1, 0])
        emcee_df.b.hist(ax = ax[1, 1])
        
        plt.show()
    
    if diagnose == True:
        xarr = np.linspace(emcee_wave[0], emcee_wave[-1], 100)
        
        plt.figure()
        plt.title('Input Emcee Spectra and Emcee Fit')
        plt.plot(emcee_wave, emcee_spec, color = 'black', alpha = 0.5, label = 'Data')
        plt.scatter(emcee_wave, emcee_spec, color = 'black')
        plt.plot(xarr, line_model(xarr, *emcee_df.quantile(q = 0.5).values[:-2]), label = 'Model')
        plt.xlabel(r'Wavelength [$\mu$m]')
        plt.ylabel('Flux')
        plt.legend()
        plt.show()
    
    if diagnose == True:
        plt.figure()
        #xarr = np.linspace(emcee_wave[0], emcee_wave[-1], 100)
        plt.title('Residual (Data - Model)')
        #plt.plot(emcee_wave, emcee_spec, color = 'black', alpha = 0.5, label = 'Data')
        #plt.scatter(emcee_wave, emcee_spec, color = 'black')
        plt.plot(emcee_wave, line_model(emcee_wave, *emcee_df.quantile(q = 0.5).values[:-2])-emcee_spec, label = 'Model')
        plt.xlabel(r'Wavelength [$\mu$m]')
        plt.ylabel('Residual Flux')
        plt.legend()
        print(np.abs( np.mean(line_model(emcee_wave, *emcee_df.quantile(q = 0.5).values[:-2])-emcee_spec)))
        plt.show()
    ###########
    #NOTE:
    #need to also give the filename argument otherwise it will overwrite the default file
    ###########
    if save == True:
        emcee_df.to_csv(filename, sep = ' ')
        
    else:
        return emcee_df

In [138]:
OIII4960 = 4958.911
OIII5007 = 5006.843
OIII4363 = 4363.210
h_beta = 4861.333
h_alpha = 6562.819
SII6716 = 6716.440
SII6731 = 6730.810
OII3726= 3726.032
OII3729= 3728.815

In [139]:
RUBIES_and_CEERS_table = Table.read('MASTER_RUBIES_AND_CEERS_TABLE.fits')
redshift = RUBIES_and_CEERS_table['redshift']
flux = RUBIES_and_CEERS_table['flux']
source_name = RUBIES_and_CEERS_table['filename']
flux_error = RUBIES_and_CEERS_table['flux_error']
rest_frame = RUBIES_and_CEERS_table['rest_frame_wavelength']
flags = RUBIES_and_CEERS_table['flags']
obs_wavelength = RUBIES_and_CEERS_table['wavelength']
which_catalog = RUBIES_and_CEERS_table['catalog']
wavelength_ang = RUBIES_and_CEERS_table['obs_wavelength_in_ang']
flux_lambda = RUBIES_and_CEERS_table['flux_lambda']
rest_frame_in_ang = RUBIES_and_CEERS_table['rest_frame_wavelength_in_ang']
flux_lambda_cleaned_for_each_spectra = RUBIES_and_CEERS_table['flux_lambda_cleaned']

## function to covert to F𝜆

In [83]:
def flux_to_flux_lambda(flux_input, wavelength_input):
    flux_input = flux_input * (10**-23)
    c = 2.99702547 * (10 **18) #angstorm / sec
    conv_factor = c/(wavelength_input**2)
    return (conv_factor*flux_input)

new_flux_master = []
for f,w in zip(flux,wavelength_ang):
    new_flux = flux_to_flux_lambda(f,w)
    new_flux_master.append(new_flux)


RUBIES_and_CEERS_table['flux_lambda'] = new_flux_master

## add catalog type as column 

In [82]:
which_catalog = []
for x in source_name:
    if 'nirspec' in x:
        which_catalog.append('CEERS')
    else:
        which_catalog.append('RUBIES')
RUBIES_and_CEERS_table['catalog'] = which_catalog

In [88]:
RUBIES_and_CEERS_table['obs_wavelength_in_ang'] = RUBIES_and_CEERS_table['wavelength']*10000
RUBIES_and_CEERS_table['rest_frame_wavelength_in_ang'] = 10000* RUBIES_and_CEERS_table['rest_frame_wavelength']

In [107]:
RUBIES_and_CEERS_table.write('MASTER_RUBIES_AND_CEERS_TABLE.fits',overwrite=True)

## pipeline to clean spectra 

In [110]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt


# Adjust flux_lambda by dividing by (1 + redshift) for each source
adjusted_flux = [flux_lambda[i] / (1 + redshift[i]) for i in range(len(flux_lambda))]

# Define the wavelength range for the zoom
zoom_min, zoom_max = 5000, 5015

# Initialize a list to hold the modified flux data
modified_flux = []

# Process each source
for i in range(len(adjusted_flux)):
    # Find the indices where the rest frame wavelength is within the zoom range
    zoom_indices = np.where((rest_frame_in_ang[i] >= zoom_min) & (rest_frame_in_ang[i] <= zoom_max))[0]

    if len(zoom_indices) > 0:
        # Extract the flux values within this range
        zoom_flux = adjusted_flux[i][zoom_indices]

        # Find the maximum flux in this zoom range
        max_flux = np.max(zoom_flux)

    else:
        # If no data in the zoom range, continue without changing the flux
        max_flux = np.nan

    # Set all flux values higher than the max in the zoom range to NaN across the entire spectrum
    flux_to_modify = np.copy(adjusted_flux[i])
    flux_to_modify[flux_to_modify > max_flux] = np.nan
    flux_to_modify[flux_to_modify < -max_flux] = np.nan
    # Append the modified flux to the list
    modified_flux.append(flux_to_modify)

# index = 5  
# plt.figure(figsize=(10, 6))
# plt.plot(rest_frame_in_ang[index], modified_flux[index],c='red')
# plt.xlabel('Rest Frame Wavelength (Angstroms)')
# plt.ylabel('Flux / (1 + Redshift)')
# #plt.xlim([4800, 5200])  
# plt.axvline(5007,c='k',alpha=0.2)

# plt.show()


note that this above pipeline has been done for all sources in both CEERS and RUBIES. it cleans the spectra such that 5007 is the highest point and anything above that or below the inverse of it goes to nan. this is to clean the spectra's noise/outliers. note also that the flux used is f_lambda converted flux plus it converted to f_lambda/(1+z) such that the ingertral of flux is now consistent with any changes done to obs_wavelength to get to rest_frame_wavelength. 

In [105]:
# RUBIES_and_CEERS_table['flux_lambda_cleaned'] = modified_flux
# RUBIES_and_CEERS_table.write('MASTER_RUBIES_AND_CEERS_TABLE.fits',overwrite=True)

In [117]:
# source = 'hlsp_ceers_jwst_nirspec_nirspec10-000618_comb-mgrat_v0.7_x1d-masked.fits'



# indx = np.where(source_name==source)[0][0]
# mask = ((np.isfinite(before_flux) )) & ((np.isfinite(before_flux_error) )) 

# Hbeta_fit = emcee_fit(rest_frame[indx][mask]*10000, flux[indx][mask], flux_error[indx][mask], h_beta, window_wavelength,diagnose= False,save=False)
# Halpha_fit = emcee_fit(rest_frame[indx][mask]*10000, flux[indx[mask]], flux_error[indx][mask], h_alpha, window_wavelength,diagnose= False,save=False)
        
# plt.figure()
# plt.step(rest_frame[indx]*10000,flux[indx])
# plt.xlim(4800,5000)
# plt.axvline(4861)

# plt.figure()
# plt.step(rest_frame[indx]*10000,flux[indx])
# plt.xlim(6400,6600)
# plt.axvline(6564)

# plt.show()